PARTE 2

In [ ]:
# 1. Crear el dataset con diagnosticos medicos

In [55]:
import pandas as pd

# Primero se crea los datos simulados
data = {
    'ID': [1, 2, 3, 4],
    'nombre_completo': ['Nelcy Avila', 'Maria Maza', 'Luis Morocho', 'Fer Avila'],
    'correo': ['nnavila@gmail.com', 'marmaz@hotmail.com', 'luismoro@gmail.com', 'feravila@gmail.com'],
    'edad': [26, 24, 52, 17],
    'diagnóstico': ['Neumonia', 'Obesidad', 'Artritis','Diabetes'],
    'contraseña': ['Nel25A', 'marmaza24', '52luisM', 'Avila17']
}

# hay que crear el DataFrame y luego se guarda
df = pd.DataFrame(data)
df.to_csv('datos_simulados.csv', index=False)

# Se muestra el contenido del dataset
df

,ID,nombre_completo,correo,edad,diagnóstico,contraseña
0,1,Nelcy Avila,nnavila@gmail.com,26,Neumonia,Nel25A
1,2,Maria Maza,marmaz@hotmail.com,24,Obesidad,marmaza24
2,3,Luis Morocho,luismoro@gmail.com,52,Artritis,52luisM
3,4,Fer Avila,feravila@gmail.com,17,Diabetes,Avila17


#Cifrado
Cifrado de campos sensibles con  Fernet.

In [56]:
from cryptography.fernet import Fernet
#teniendo el archivo realizado anteriormente:
df = pd.read_csv('datos_simulados.csv')

# Se genera una clave y posteriormente se le guarda
clave = Fernet.generate_key()
with open('clave.key', 'wb') as file:
    file.write(clave)

fernet = Fernet(clave)

# Ciframos elcorreo y también el diagnóstico
df['correo_cifrado'] = df['correo'].apply(lambda x: fernet.encrypt(x.encode()).decode())
df['diagnóstico_cifrado'] = df['diagnóstico'].apply(lambda x: fernet.encrypt(x.encode()).decode())

# Hay que eliminar los campos originales
df.drop(['correo', 'diagnóstico'], axis=1, inplace=True)

# Guardar el resultado de los datos cifrados
df.to_csv('datos_cifrados.csv', index=False)

# Se ve el resultado después de haber cifrado los campos sensibles, es decir el correo y diagnóstico.
df


,ID,nombre_completo,edad,contraseña,correo_cifrado,diagnóstico_cifrado
0,1,Nelcy Avila,26,Nel25A,gAAAAABoYdVVxgXbgpxZOPFGl4CJb5gfafBHWV-v-baC69...,gAAAAABoYdVVFgVyTEoLEfPpF5BLZltEoyhHUGFEljQjlv...
1,2,Maria Maza,24,marmaza24,gAAAAABoYdVV5SHdFj27fdz5BMMUM5-9cQSw-C-__EcTqm...,gAAAAABoYdVVPC3dlIArGX1GG5gcu4-7-yZdBUk48QpGGF...
2,3,Luis Morocho,52,52luisM,gAAAAABoYdVV7DhmgiaJjoFZu6nAiJCqMbUcukQ5Y73aDm...,gAAAAABoYdVVLeW8WbK2i2x49gA4lFnWY_znXLvo9mV9v4...
3,4,Fer Avila,17,Avila17,gAAAAABoYdVVUmLS6Kx7w9mRLSrv8ec2VpndSXXD1jBoa1...,gAAAAABoYdVVhHI8cO11KAS81BXvMpmI0nXu-8sdjnmpqY...


#Pseudonimización:
Pseudonimización de nombre_completo y correo usando hashlib.

In [57]:
import hashlib

# Se pseudonimiza el nombre y correo cifrado
df['nombre_pseudo'] = df['nombre_completo'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())
df['correo_pseudo'] = df['correo_cifrado'].apply(lambda x: hashlib.sha256(x.encode()).hexdigest())

# Aquí se eliminan los campos originales, en este caso, los nombres delos pacientes y sus correos
df.drop(['nombre_completo', 'correo_cifrado'], axis=1, inplace=True)

# Se muestran los resultado con pseudonimización de los nombres y los correos de los pacientes
df


,ID,edad,contraseña,diagnóstico_cifrado,nombre_pseudo,correo_pseudo
0,1,26,Nel25A,gAAAAABoYdVVFgVyTEoLEfPpF5BLZltEoyhHUGFEljQjlv...,1ec7bad7c211622db17fd4d8f5ec80f7d41f24a895e8df...,9c65fbf9c520b957d56f3cdddb59b89ac50d64e47f71c4...
1,2,24,marmaza24,gAAAAABoYdVVPC3dlIArGX1GG5gcu4-7-yZdBUk48QpGGF...,546d9aad1d55e737929b709976a3a4d84ff043a15fa0b6...,9a6db4ec7656fd1f3bf88db001eb0991cd1a3dc7dd832d...
2,3,52,52luisM,gAAAAABoYdVVLeW8WbK2i2x49gA4lFnWY_znXLvo9mV9v4...,e547bb18a132aea01772294e91c960a7afb59adcbd6db0...,931ab6757ea840cfaeeee6a45243348dda8986f5e514e7...
3,4,17,Avila17,gAAAAABoYdVVhHI8cO11KAS81BXvMpmI0nXu-8sdjnmpqY...,e4843f0f913ffc0d3c1627ccc80b46af0a03ace836d460...,342445353bc56b0e19d2495263c1cca8ca15862c0ad4f2...


#Hash seguro
Hash seguro de contraseñas con bcrypt

In [58]:
!pip install bcrypt


In [59]:
import bcrypt

# Función para hacer el hash de las contraseñas
def hashear_contraseña(pw):
    return bcrypt.hashpw(pw.encode(), bcrypt.gensalt()).decode()

# Aplicar hash a cada contraseña
df['contraseña_hash'] = df['contraseña'].apply(hashear_contraseña)

# Se elimina la contraseña original de cada paciente
df.drop(['contraseña'], axis=1, inplace=True)

# Luego guradamos el archivo final con seguridad
df.to_csv('datos_seguridad.csv', index=False)

# Finalmente mostramos el resultado final con contraseñas con seguro en cada uno de los pacientes
df


,ID,edad,diagnóstico_cifrado,nombre_pseudo,correo_pseudo,contraseña_hash
0,1,26,gAAAAABoYdVVFgVyTEoLEfPpF5BLZltEoyhHUGFEljQjlv...,1ec7bad7c211622db17fd4d8f5ec80f7d41f24a895e8df...,9c65fbf9c520b957d56f3cdddb59b89ac50d64e47f71c4...,$2b$12$ysz8uAWiCWR2/CKUAW0LjO.nRLgKC9SX7c7qP19...
1,2,24,gAAAAABoYdVVPC3dlIArGX1GG5gcu4-7-yZdBUk48QpGGF...,546d9aad1d55e737929b709976a3a4d84ff043a15fa0b6...,9a6db4ec7656fd1f3bf88db001eb0991cd1a3dc7dd832d...,$2b$12$/0cGuJQNtmOWlVwHe39L9ewSH9X7IB5EjkHthQS...
2,3,52,gAAAAABoYdVVLeW8WbK2i2x49gA4lFnWY_znXLvo9mV9v4...,e547bb18a132aea01772294e91c960a7afb59adcbd6db0...,931ab6757ea840cfaeeee6a45243348dda8986f5e514e7...,$2b$12$EpMw9SGKKQIuGfp2Bo76mOPhiLBzYUqnvSwnAMH...
3,4,17,gAAAAABoYdVVhHI8cO11KAS81BXvMpmI0nXu-8sdjnmpqY...,e4843f0f913ffc0d3c1627ccc80b46af0a03ace836d460...,342445353bc56b0e19d2495263c1cca8ca15862c0ad4f2...,$2b$12$LZ/moXA1.9KR500GdB1UAeAKBeeRiDJDACIsJ2B...


# **Control de acceso simulado**

In [60]:
# Control de acceso simple
clave_segura = "accesopaciente2025"

entrada = input("Por favor, introduzca la contraseña para acceder a los datos: ")
if entrada == clave_segura:
    print("Usted ha accedido a la base de datos de los pacientes")

    print(df)
else:
    print("¡Acceso denegado, contraseña incorrecta!")


Por favor, introduzca la contraseña para acceder a los datos: accesopaciente2025
Usted ha accedido a la base de datos de los pacientes
   ID  edad                                diagnóstico_cifrado  \
0   1    26  gAAAAABoYdVVFgVyTEoLEfPpF5BLZltEoyhHUGFEljQjlv...   
1   2    24  gAAAAABoYdVVPC3dlIArGX1GG5gcu4-7-yZdBUk48QpGGF...   
2   3    52  gAAAAABoYdVVLeW8WbK2i2x49gA4lFnWY_znXLvo9mV9v4...   
3   4    17  gAAAAABoYdVVhHI8cO11KAS81BXvMpmI0nXu-8sdjnmpqY...   

                                       nombre_pseudo  \
0  1ec7bad7c211622db17fd4d8f5ec80f7d41f24a895e8df...   
1  546d9aad1d55e737929b709976a3a4d84ff043a15fa0b6...   
2  e547bb18a132aea01772294e91c960a7afb59adcbd6db0...   
3  e4843f0f913ffc0d3c1627ccc80b46af0a03ace836d460...   

                                       correo_pseudo  \
0  9c65fbf9c520b957d56f3cdddb59b89ac50d64e47f71c4...   
1  9a6db4ec7656fd1f3bf88db001eb0991cd1a3dc7dd832d...   
2  931ab6757ea840cfaeeee6a45243348dda8986f5e514e7...   
3  342445353bc56b0e19d2495263